In [ ]:
import numpy as np
import uproot as uproot
import uproot3 as uproot3
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from tqdm.notebook import tqdm

import pickle

from scipy.stats import chi2


In [ ]:
f_merged = uproot.open("numuCC_3d_data/real_data/merge_xs_data.root")
f_wiener = uproot.open("numuCC_3d_data/real_data/wiener_data.root")

In [ ]:
mc_sig_pred = []
mc_bkg_pred = []
mc_sig_plus_bkg_pred = []
mc_tot_pred = []
ext_pred = []
tot_pred = []
data = []
tot_pred_from_hmc = []

# 216 total channels, 72 sig, 72 bkg, 72 EXT

for i in range(72):
    mc_sig_pred += list(f_merged[f"histo_{i+1}"].values(flow=True)[1:])
    mc_bkg_pred += list(f_merged[f"histo_{i+1 + 72}"].values(flow=True)[1:])
    ext_pred += list(f_merged[f"histo_{i+1 + 72*2}"].values(flow=True)[1:])

    mc_sig_plus_bkg_pred += list(f_merged[f"histo_{i+1}"].values(flow=True)[1:] + f_merged[f"histo_{i+1 + 72}"].values(flow=True)[1:])
        
    tot_pred += list(f_merged[f"histo_{i+1}"].values(flow=True)[1:] + f_merged[f"histo_{i+1 + 72}"].values(flow=True)[1:] + f_merged[f"histo_{i+1 + 72*2}"].values(flow=True)[1:])
    
    tot_pred_from_hmc += list(f_merged[f"hmc_obsch_{i+1}"].values(flow=True)[1:])
    
    data += list(f_merged[f"hdata_obsch_{i+1}"].values(flow=True)[1:])

print("total data, pred: ", sum(data), sum(tot_pred))
print("total FC data, pred: ", sum(data[:16*9*4]), sum(tot_pred[:16*9*4]))
print("total PC data, pred: ", sum(data[16*9*4:]), sum(tot_pred[16*9*4:]))




In [ ]:
mat_collapse = f_merged["mat_collapse"].member("fElements").reshape((3456, 1152))

cov_17 = uproot.open("numuCC_3d_data/nuwro_fake_data/XsFlux/cov_17.root")
cov_17_vec_mean = mat_collapse.T @ cov_17["vec_mean_17"].member("fElements")
cov_17_arr_uncollapsed = cov_17["cov_xf_mat_17"].member("fElements").reshape((3456, 3456))
cov_17_arr = mat_collapse.T @ cov_17_arr_uncollapsed @ mat_collapse

frac_cov_17 = cov_17_arr / np.outer(cov_17_vec_mean, cov_17_vec_mean)

cov_xs = frac_cov_17 * np.outer(tot_pred, tot_pred)

# this removes XS uncertainty that cancels out in the extracted XS, not full uncertainties
#cov_xs = f_wiener["hcov_xs"].to_numpy()[0]


In [ ]:
#cov_stat = f_wiener["hcov_stat"].to_numpy()[0] # not using this, data stat is separate
cov_mcstat = f_wiener["hcov_mcstat"].to_numpy()[0]
cov_add = f_wiener["hcov_add"].to_numpy()[0]
cov_det = f_wiener["hcov_det"].to_numpy()[0]
cov_flux = f_wiener["hcov_flux"].to_numpy()[0]
#cov_tot = f_wiener["hcov_tot"].to_numpy()[0] # not using this, data stat is separate

total_pred_cov = (cov_xs
        + cov_mcstat
        + cov_add
        + cov_det
        + cov_flux
        )

In [ ]:
16*9*4*2

In [ ]:
momentum_bins = np.linspace(0, 1.6, 17)
momentum_bin_centers = (momentum_bins[1:] + momentum_bins[:-1]) / 2

fig, axs = plt.subplots(3, 3, figsize=(15, 15))


for fc_pc in ["FC", "PC"]:

    if fc_pc == "FC":
        ls = "solid"
        marker = "o"
        hatch = None
        fc_pc_start_bin_index = 0
    else:
        ls = "dashed"
        marker = "x"
        hatch = "///"
        fc_pc_start_bin_index = 16*9*4

    for energy_bin in range(4):

        c = ["tab:blue", "tab:orange", "tab:green", "tab:red"][energy_bin]

        energy_start_bin_index = fc_pc_start_bin_index + 16*9*energy_bin

        for theta_bin in range(9):

            theta_start_bin_index = energy_start_bin_index + 16*theta_bin

            ax = axs[theta_bin // 3, theta_bin % 3]

            curr_pred = tot_pred[theta_start_bin_index:theta_start_bin_index+16]
            curr_pred_err = np.sqrt(np.diag(total_pred_cov)[theta_start_bin_index:theta_start_bin_index+16])
            upper_pred = curr_pred + curr_pred_err
            lower_pred = curr_pred - curr_pred_err

            curr_data = data[theta_start_bin_index:theta_start_bin_index+16]

            ax.plot(momentum_bin_centers, curr_pred, c=c, ls=ls)
            
            ax.fill_between(momentum_bin_centers, lower_pred, upper_pred, color=c, alpha=0.2, hatch=hatch)
            #ax.scatter(momentum_bin_centers, data[theta_start_bin_index:theta_start_bin_index+16], c=c, marker=marker)
            ax.errorbar(momentum_bin_centers, curr_data, yerr=np.sqrt(curr_data), fmt=marker, c=c, capsize=3, capthick=2)

fc_pred, = axs[0][2].plot([], [], c="k", ls="solid")
pc_pred, = axs[0][2].plot([], [], c="k", ls="dashed")
fc_data = axs[0][2].scatter([], [], c="k", marker="o")
pc_data = axs[0][2].scatter([], [], c="k", marker="x")
fc_err = axs[0][2].fill_between([], [], [], color="k", alpha=0.2, hatch=None)
pc_err = axs[0][2].fill_between([], [], [], color="k", alpha=0.2, hatch="///")

e1, = axs[0][2].plot([], [], c="tab:blue", ls="-", label=r"0.2 < $E_\nu$ < 0.705 GeV")
e2, = axs[0][2].plot([], [], c="tab:orange", ls="-", label=r"0.705 < $E_\nu$ < 1.05 GeV")
e3, = axs[0][2].plot([], [], c="tab:green", ls="-", label=r"1.05 < $E_\nu$ < 1.57 GeV")
e4, = axs[0][2].plot([], [], c="tab:red", ls="-", label=r"1.57 GeV < $E_\nu$")

enu_legend = axs[0][2].legend(loc="right", fontsize=10, bbox_to_anchor=(1,0.7), bbox_transform=axs[0][2].transAxes)
axs[0, 2].add_artist(enu_legend)

from matplotlib.legend_handler import HandlerTuple
l = axs[0][2].legend([(fc_pred, fc_err, fc_data), (pc_pred, pc_err, pc_data)], ['FC pred, err, data', 'PC pred, err, data'], handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper right")

theta_bins = [-1, -0.5, 0, 0.27, 0.45, 0.62, 0.76, 0.86, 0.94, 1]
energy_bins = [200, 705, 1050, 1570, 4000]

for i, ax in enumerate(axs.flatten()):
    ax.set_yscale("symlog", linthresh=1)
    ax.set_ylim(-0.1, 1e4)
    ax.set_xlim(0, 1.6)

    ax.text(0.03, 0.9, rf"{theta_bins[i]}$ < \cos \theta_\mu < $ {theta_bins[i+1]}", transform=ax.transAxes)


fig.suptitle(r"All $\nu_\mu$CC Inclusive Data and Predictions")

for ax in axs[-1]:
    ax.set_xlabel(r"$P_\mu$ (GeV/c)")
for ax in axs[:,0]:
    ax.set_ylabel("Events / bin")

for ax in axs[0, :]:
    ax.get_xaxis().set_visible(False)
for ax in axs[1, :]:
    ax.get_xaxis().set_visible(False)
for ax in axs[:, 1]:
    ax.get_yaxis().set_visible(False)
for ax in axs[:, 2]:
    ax.get_yaxis().set_visible(False)
    
            
plt.tight_layout()

plt.savefig("plots/all_data_and_predictions.png", facecolor="white", dpi=300)
plt.savefig("plots/all_data_and_predictions.jpg", facecolor="white", dpi=300)
plt.savefig("plots/all_data_and_predictions.svg", facecolor="white")

